In [1]:
import pandas as pd
import json
from superlinked import framework as sl
import sys
sys.path.append('../code/')
from utills import load_data, build_superlinked_app
import umap
import altair as alt

2025-04-07 17:12:18.303 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
df = load_data(sample_size=1000)
app, index, food_item, description_space, food_category_text_space, food_category_categorical_space, energy_space = build_superlinked_app(df)


2025-04-07 15:05:42.939 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-04-07 15:05:42.940 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-07 15:05:43.056 
  command:

    streamlit run /home/ec2-user/projects/PhenoDS/superlinked_demo/venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-07 15:05:43.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-07 15:05:43.058 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-07 15:05:43.086 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-07 15:05:43.086 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [4]:
# collect all vectors from the app
vs = sl.VectorSampler(app=app)
vector_collection = vs.get_all_vectors(index, food_item)
vectors = vector_collection.vectors
vector_df = pd.DataFrame(vectors, index=[int(id_) for id_ in vector_collection.id_list])
vector_df.head()
umap_transform = umap.UMAP(random_state=0, transform_seed=0, n_jobs=1, metric="cosine")
umap_transform = umap_transform.fit(vector_df)
umap_vectors = umap_transform.transform(vector_df)
umap_df = pd.DataFrame(umap_vectors, columns=["dimension_1", "dimension_2"], index=vector_df.index)
umap_df = umap_df.join(df.set_index('fdc_id')[['description', 'food_category', 'calories']])

/home/ec2-user/projects/PhenoDS/superlinked_demo/venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/ec2-user/projects/PhenoDS/superlinked_demo/venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [6]:






umap_df.head()



import altair as alt
alt.Chart(umap_df.head(10)).mark_circle(size=8).encode(x="dimension_1", y="dimension_2", color="food_category:N").properties(
    width=600, height=500, title="UMAP Transformed vectors coloured by food category"
).configure_title(
    fontSize=16,
    anchor="middle",
).configure_legend(
    strokeColor="black",
    padding=10,
    cornerRadius=10,
    labelFontSize=14,
    titleFontSize=14,
).configure_axis(
    titleFontSize=14, labelFontSize=12
)

alt.Chart(...)